In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import scipy.stats as stats
import datetime
import openpyxl

In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 24.7 MB/s eta 0:00:00


In [ ]:
from urllib.parse import quote_plus
USERNAME_RAW = "Ninh_DoAn"
PASSWORD_RAW = "123456Ninh"
HOSTNAME_URI_PART = "ninhbeta.8kk4yvt.mongodb.net"
APP_NAME = "ninhbeta"
USERNAME_ENCODED = quote_plus(USERNAME_RAW)
PASSWORD_ENCODED = quote_plus(PASSWORD_RAW)
MONGO_URI = f"mongodb+srv://{USERNAME_ENCODED}:{PASSWORD_ENCODED}@{HOSTNAME_URI_PART}/?appName={APP_NAME}"
print("Chuỗi kết nối đã được mã hóa hoàn chỉnh.")

Chuỗi kết nối đã được mã hóa hoàn chỉnh.


In [ ]:
import pymongo
from pymongo.mongo_client import MongoClient
try:
    client = MongoClient(MONGO_URI)
    client.admin.command('ping')
    db = client["DoAn_Congtacnhom"]
    print("KẾT NỐI THÀNH CÔNG VÀ SẴN SÀNG SỬ DỤNG!")
except Exception as e:
    print(f"Lỗi kết nối: {e}")

KẾT NỐI THÀNH CÔNG VÀ SẴN SÀNG SỬ DỤNG!


#Thành viên

In [ ]:
def import_excel_to_mongo(excel_file_path, Thanh_vien):
    try:
        df = pd.read_excel('/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/Thanh_vien.xlsx')
        data = df.to_dict('records')
        ngay_hien_tai = datetime.datetime.now()
        for item in data:
            item['NgayTG'] = ngay_hien_tai
        col = db[Thanh_vien]
        col.delete_many({})
        if data:
            col.insert_many(data)
            print(f"Đã nhập thành công {len(data)} dòng từ '{excel_file_path}' vào collection '{Thanh_vien}'")
        else:
            print("File excel trống!")
    except Exception as e:
        print(f"Lỗi: {e}")
import_excel_to_mongo('Thanh_vien.csv', 'Thanh_vien')

Đã nhập thành công 15 dòng từ 'Thanh_vien.csv' vào collection 'Thanh_vien'


#Nhóm

In [ ]:
def import_groups_excel(file_path, collection_name='groups'):
    try:
        df = pd.read_excel('/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/Group.xlsx', dtype={'Thanh_vien': str})
        data_raw = df.to_dict('records')
        data_clean = []
        users_col = db['Thanh_vien']
        for item in data_raw:
            if 'NgayTao' in item:
                 item['NgayTao'] = pd.to_datetime(item['NgayTao'], errors='coerce')
            member_str = item.get('Thanh_vien', '')
            detailed_members_list = []
            if pd.notna(member_str) and str(member_str).strip() != '':
                id_list = [x.strip() for x in str(member_str).split(',')]

                # --- TRA CỨU USER ---
                for user_id in id_list:
                    # Tìm user trong database dựa theo ID
                    user_info = users_col.find_one({"_id": user_id})
                    if user_info:
                        detailed_members_list.append({
                            "id": user_id,
                            "HoTen": user_info.get('HoTen', 'Chưa cập nhật tên'),
                            "role_id": user_info.get('role_id', 'Member')
                        })
                    else:
                        detailed_members_list.append({
                            "id": user_id,
                            "HoTen": "Không tìm thấy",
                            "role_id": "Unknown"
                        })
            item['Thanh_vien'] = detailed_members_list
            data_clean.append(item)
        col = db[collection_name]
        col.delete_many({})
        if data_clean:
            col.insert_many(data_clean)
            print(f"Nhóm Đã nhập {len(data_clean)} dòng từ '{file_path}'.")
        else:
            print(f"File '{file_path}' trống!")
    except Exception as e:
        print(f"Lỗi nhập Nhóm: {e}")
import_groups_excel('Group.xlsx', 'groups')


Nhóm Đã nhập 3 dòng từ 'Group.xlsx'.


#Dự án

In [ ]:
def import_DuAn_excel(excel_file_path):
    try:
        df = pd.read_excel('/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/DuAn.xlsx', dtype={'group_id': str})
        df['NgayBD'] = pd.to_datetime(df['NgayBD'], format='%Y-%m-%d', errors='coerce')
        df['NgayKT'] = pd.to_datetime(df['NgayKT'], format='%Y-%m-%d', errors='coerce')
        data = df.to_dict('records')
        col = db['projects']
        col.delete_many({})
        if data:
            col.insert_many(data)
            print(f"Đã nhập thành công {len(data)} dự án từ '{excel_file_path}'")
        else:
            print("File CSV trống!")
    except Exception as e:
        print(f"Lỗi: {e}")
import_DuAn_excel('DuAn.xlsx')

Đã nhập thành công 3 dự án từ 'DuAn.xlsx'


#Kiểm tra NaN,Null .Xem trước dữ liệu, xóa dcm cũ để tránh lỗi

In [ ]:
def simple_import_sheet(file_path, sheet_name, collection_name, column_dtypes=None):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, dtype=column_dtypes)
        df = df.fillna(value=np.nan).replace({np.nan: None})
        print(f"\n--- Kiểm tra 5 dòng đầu tiên của sheet '{sheet_name}' (Null = None): ---")
        print(df.head())
        print("-----------------------------------------------------------------------------------")
        data = df.to_dict('records')
        ngay_hien_tai = datetime.datetime.now()
        for item in data:
            item['NgayImport'] = ngay_hien_tai
        col = db[collection_name]
        delete_result = col.delete_many({})
        if data:
            col.insert_many(data)
            print(f"Đã xóa {delete_result.deleted_count} documents cũ.")
            print(f"Đã nhập thành công {len(data)} dòng từ sheet '{sheet_name}' vào collection '{collection_name}'")
        else:
            print("Sheet excel trống!")
    except Exception as e:
        print(f"Lỗi khi xử lý sheet '{sheet_name}': {e}")

#Taks

In [ ]:
excel_path = "/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/taks.xlsx"
file_name = "taks.xlsx"
taks_dtypes = {
    "_id": str,
    "TenTask": str,
    "MoTa_Ta": str,
    "ID_ThanhVien": str,
    "ID_DuAn": str
}
print(f"--- Bắt đầu Import Sheet: Taks từ {file_name} ---")
simple_import_sheet(
    file_path=excel_path,
    sheet_name="Taks",
    collection_name="Tasks",
    column_dtypes=taks_dtypes
)

--- Bắt đầu Import Sheet: Taks từ taks.xlsx ---

--- Kiểm tra 5 dòng đầu tiên của sheet 'Taks' (Null = None): ---
   _id                           TenTask  \
0  TO1                    Khảo sát dự án   
1  TO2           Phân tích sơ đồ dữ liệu   
2  TO3  Xác định các Ràng buộc & Quy tắc   
3  TO4              Thu thập dữ liệu mẫu   
4  TO5                  Làm sạch dữ liệu   

                                             MoTa_Ta Trangthai     NgayBD  \
0  Phỏng vấn, thu thập các quy trình quản lý ký t...      done 2025-11-08   
1  Thiết kế các bảng, mối quan hệ giữa các thực t...      done 2025-11-09   
2  Định nghĩa các quy tắc kinh doanh cụ thể. Ví d...      done 2025-11-09   
3  Thu thập dữ liệu thực tế (hoặc giả lập) để phụ...      done 2025-11-10   
4  Chuẩn hóa định dạng, xử lý dữ liệu thiếu hoặc ...      done 2025-11-22   

      NgayKT ID_ThanhVien ID_DuAn  
0 2025-11-20      AO2,AO5     CO1  
1 2025-11-20          AO2     CO1  
2 2025-11-10          AO2     CO1  
3 2025-11-21  

#File

In [ ]:
excel_path = "/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/taks.xlsx"
file_dtypes = {
    "_id": str,
    "TenFile": str,
    "ID_Taks": str,
    "LoaiFile": str,
    "DuongDan": str,
    "ID_NguoiGui": str
}
print("\n--- Bắt đầu Import Sheet: File ---")
simple_import_sheet(
    file_path=excel_path,
    sheet_name="File",
    collection_name="Files",
    column_dtypes=file_dtypes
)


--- Bắt đầu Import Sheet: File ---

--- Kiểm tra 5 dòng đầu tiên của sheet 'File' (Null = None): ---
   _id           TenFile     NgayUp ID_Taks Loai_File ID_NguoiGui  \
0  FO1  File_test_dulieu 2025-12-06     TO8      taks         AO2   
1  FO2         DuAn_Full 2026-01-01    TO15      taks         AO3   
2  FO3       File_YeuCau 2025-11-08    TO17      taks    AO6,AO10   
3  FO4    File_CleanData 2025-11-24    TO20      taks        AO10   
4  FO5  File_test_nhatro 2025-12-04    TO23      taks        AO10   

                                            DuongDan  
0  https://docs.google.com/document/d/1gFePtyIw40...  
1  https://docs.google.com/document/d/1gFePtyIw40...  
2  https://docs.google.com/document/d/1gFePtyIw40...  
3  https://docs.google.com/document/d/1gFePtyIw40...  
4  https://docs.google.com/document/d/1gFePtyIw40...  
-----------------------------------------------------------------------------------
Đã xóa 9 documents cũ.
Đã nhập thành công 9 dòng từ sheet 'File' vào 

#Comments

In [ ]:
excel_path = "/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/taks.xlsx"
comments_dtypes = {
    "_id": str,
    "ID_Taks": str,
    "NoiDung": str,
    "ID_Thanhvien": str
}
print("\n--- Bắt đầu Import Sheet: Comments ---")
simple_import_sheet(
    file_path=excel_path,
    sheet_name="Comments",
    collection_name="Comments",
    column_dtypes=comments_dtypes
)


--- Bắt đầu Import Sheet: Comments ---

--- Kiểm tra 5 dòng đầu tiên của sheet 'Comments' (Null = None): ---
    _id                                      NoiDung     NgayCM ID_Taks  \
0  CMO1            Test dự liệu lỗi quá lỗi rồi team 2025-12-03     TO7   
1  CMO2  Test dự liệu thành công không phát hiện lỗi 2025-12-06     TO7   
2  CMO3          Chọn framework với ngôn ngữ nào đây 2025-12-07     TO9   
3  CMO4                     Giờ chọn trọ ai hỏi đây? 2025-11-08    TO17   
4  CMO5                Lỗi kết nối database rồi team 2025-12-03    TO13   

  ID_Thanhvien  
0          AO5  
1          AO5  
2          AO3  
3          AO3  
4          AO1  
-----------------------------------------------------------------------------------
Đã xóa 10 documents cũ.
Đã nhập thành công 14 dòng từ sheet 'Comments' vào collection 'Comments'


#Lịch sử hoạt động

In [ ]:
excel_path = "/content/drive/MyDrive/DoAnCoSoNganh_Ninh_DA23TTA/taks.xlsx"
lichsu_dtypes = {
    "_id": str,
    "ID_Thanhvien": str,
    "Mo_ta":str,
    "TenHD": str
}
print("\n--- Bắt đầu Import Sheet: Lichsu_hd ---")
simple_import_sheet(
    file_path=excel_path,
    sheet_name="Lichsu_hd",
    collection_name="ActivityHistory",
    column_dtypes=lichsu_dtypes
)


--- Bắt đầu Import Sheet: Lichsu_hd ---

--- Kiểm tra 5 dòng đầu tiên của sheet 'Lichsu_hd' (Null = None): ---
    _id                                             TenHD  \
0  HDO1  Kiểm tra độ chính xác của dữ liệu (Test Dữ liệu)   
1  HDO2                        User Acceptance Test (UAT)   
2  HDO3     Kiểm tra độ chính xác của dữ liệu (Data Test)   
3  HDO4   User Acceptance Test (UAT - Kiểm thử chấp nhận)   
4  HDO5     Kiểm tra độ chính xác của dữ liệu (Data Test)   

           ThoiGian_HD                                              Mo_ta  \
0  2025-12-06 00:00:00  Kiểm tra tính toàn vẹn, tính duy nhất, và logi...   
1  2026-01-04 00:00:00                       Khoa test hệ thống lần cuối.   
2  2025-12-06 00:00:00  Kiểm tra ràng buộc dữ liệu (ví dụ: một phòng c...   
3  2026-01-04 00:00:00  Thắng thực hiện các quy trình nghiệp vụ thực t...   
4  2025-12-06 00:00:00  Kiểm tra logic giá, tính nhất quán của danh mụ...   

  ID_Thanhvien  
0          AO5  
1          AO2  
2      

#Xem thông tin của một Database cụ thể và collection bên trong nó có gì


In [ ]:
database_to_view = input("Nhập tên database: ")
print(f"\n--- {database_to_view} ---")
try:
    db = client[database_to_view]
    collection_names = db.list_collection_names()
    if not collection_names:
        print(f"Không có collection nào trong database '{database_to_view}'.")
    else:
        for col_name in collection_names:
            print(f"Collection: {col_name}")
            collection = db[col_name]
            print("Sample Documents (first 5):")
            sample_documents = collection.find().limit(5)
            found_docs = False
            for doc in sample_documents:
                print(f"      {doc}")
                found_docs = True
            if not found_docs:
                print("Không tìm thấy tài liệu nào trong collection này.")
except Exception as e:
    print(f"Lỗi khi truy cập database '{database_to_view}': {e}").pretty()

Nhập tên database: DoAn_Congtacnhom

--- Listing Collections and Sample Documents for Database: DoAn_Congtacnhom ---
Collection: projects
Sample Documents (first 5):
      {'_id': 'CO1', 'Name_pr': 'Hệ thống quản lý ký túc xá', 'group_id': 'BO1', 'NgayBD': datetime.datetime(2025, 12, 6, 0, 0), 'NgayKT': datetime.datetime(2026, 1, 10, 0, 0), 'Mo_ta': 'Xây dựng phần mềm nhằm tự động hóa và tối ưu hóa các nghiệp vụ quản lý sinh viên, phòng ốc, thu chi và an ninh trong môi trường ký túc xá.'}
      {'_id': 'CO2', 'Name_pr': 'Hệ thống quản lý nhà trọ', 'group_id': 'BO2', 'NgayBD': datetime.datetime(2025, 12, 7, 0, 0), 'NgayKT': datetime.datetime(2026, 1, 11, 0, 0), 'Mo_ta': 'Xây dựng phần mềm hỗ trợ chủ nhà hoặc đơn vị quản lý theo dõi hợp đồng thuê, quản lý thông tin khách thuê, tính toán tiền điện/nước và thu chi một cách hiệu quả cho các phòng trọ/khu trọ.'}
      {'_id': 'CO3', 'Name_pr': 'Hệ thống quản lý nhà xe sinh viên', 'group_id': 'BO3', 'NgayBD': datetime.datetime(2025, 12, 8, 0,

In [ ]:
# Tìm một gì đó (_id) có (AO1)
doc = collection.find_one({"_id": "AO1"})
print(doc)

{'_id': 'AO1', 'HoTen': 'Nguyễn An Ninh', 'Email': 'ninhnguyen@gmail.com', 'role_id': 'member', 'NgayTG': datetime.datetime(2025, 12, 14, 0, 34, 52, 722000)}


In [ ]:
def get_member_info(member_id):
    try:
        members_collection = db['Thanh_vien']
        member_document = members_collection.find_one({"_id": member_id})
        return member_document
    except Exception as e:
        print(f"Lỗi khi lấy thông tin thành viên: {e}")
        return None

def get_member_groups(member_id):
    try:
        groups_collection = db['groups']
        member_groups = list(groups_collection.find({"Thanh_vien": {"$elemMatch": {"id": member_id}}}))
        return member_groups
    except Exception as e:
        print(f"Lỗi khi lấy thông tin nhóm của thành viên: {e}")
        return []

def get_projects_for_groups(group_ids):
    try:
        projects_collection = db['projects']
        related_projects = list(projects_collection.find({"group_id": {"$in": group_ids}}))
        return related_projects
    except Exception as e:
        print(f"Lỗi khi lấy thông tin dự án cho các nhóm: {e}")
        return []

def get_member_tasks(member_id):
    try:
        tasks_collection = db['Tasks']
        regex_pattern = f".*(^|,){member_id}(,|$).*"
        member_tasks = list(tasks_collection.find({"ID_ThanhVien": {"$regex": regex_pattern}}))
        return member_tasks
    except Exception as e:
        print(f"Lỗi khi lấy thông tin nhiệm vụ của thành viên: {e}")
        return []

def get_full_member_details(member_id):
    print(f"\n--- Tổng hợp thông tin chi tiết cho thành viên: {member_id} ---")

    member_info = get_member_info(member_id)
    if member_info:
        print("\n--- Thông tin thành viên --- ")
        print(f"ID: {member_info.get('_id')}")
        print(f"Họ Tên: {member_info.get('HoTen')}")
        print(f"Email: {member_info.get('Email')}")
        print(f"Vai trò: {member_info.get('role_id')}")
        #print(f"MongoDB Query Equivalent: db.Thanh_vien.find_one({{' _id': '{member_id}'}}))")
    else:
        print(f"Không tìm thấy thông tin thành viên với ID: {member_id}")
        return

    member_groups = get_member_groups(member_id)
    if member_groups:
        print("\n--- Các nhóm thuộc về --- ")
        group_ids_for_projects = []
        for group in member_groups:
            print(f"  - Tên nhóm: {group.get('TenGr')}, ID nhóm: {group.get('_id')}")
            group_ids_for_projects.append(group.get('_id'))
        mongodb_query_string_groups = f"db.groups.find({{'Thanh_vien': {{'\\$elemMatch': {{'id': '{member_id}'}}}}}}))"
        #print(f"MongoDB Query Equivalent: {mongodb_query_string_groups}")
    else:
        print(f"Không tìm thấy nhóm nào cho thành viên với ID: {member_id}")
        group_ids_for_projects = []

    if group_ids_for_projects:
        related_projects = get_projects_for_groups(group_ids_for_projects)
        if related_projects:
            print("\n--- Các dự án liên quan --- ")
            for project in related_projects:
                print(f"  - Tên dự án: {project.get('Name_pr')}, ID dự án: {project.get('_id')}")
            group_ids_str = ', '.join([f"'{gid}'" for gid in group_ids_for_projects])
            mongodb_query_string_projects = f"db.projects.find({{'group_id': {{'\\$in': [{group_ids_str}]}}}}))"
            #print(f"MongoDB Query Equivalent: {mongodb_query_string_projects}")
        else:
            print(f"Không tìm thấy dự án nào cho các nhóm có ID: {group_ids_for_projects}")
    else:
        print("Không có nhóm nào, do đó không có dự án liên quan để hiển thị.")

    member_tasks = get_member_tasks(member_id)
    if member_tasks:
        print("\n--- Các nhiệm vụ được giao --- ")
        for task in member_tasks:
            print(f"  - Tên nhiệm vụ: {task.get('TenTask')}, ID nhiệm vụ: {task.get('_id')}")
        regex_pattern_str = f".*(^|,){member_id}(,|$).*"
        mongodb_query_string_tasks = f"db.Tasks.find({{'ID_ThanhVien': {{'\\$regex': '{regex_pattern_str}'}}}}))"
        #print(f"MongoDB Query Equivalent: {mongodb_query_string_tasks}")
    else:
        print(f"Không tìm thấy nhiệm vụ nào cho thành viên với ID: {member_id}")

print("Tất cả các hàm đã được định nghĩa.")

Tất cả các hàm đã được định nghĩa.


In [ ]:
import time
start_time = time.time()
get_full_member_details("AO13")
end_time = time.time()
print(f"Thời gian thực thi truy vấn: {end_time - start_time:.4f} giây")


--- Tổng hợp thông tin chi tiết cho thành viên: AO13 ---

--- Thông tin thành viên --- 
ID: AO13
Họ Tên: Lâm Minh Thiện
Email: minhthien@gmail.com
Vai trò: member

--- Các nhóm thuộc về --- 
  - Tên nhóm: Nhóm 3, ID nhóm: BO3

--- Các dự án liên quan --- 
  - Tên dự án: Hệ thống quản lý nhà xe sinh viên, ID dự án: CO3

--- Các nhiệm vụ được giao --- 
  - Tên nhiệm vụ: Khảo sát dự án & Phân tích yêu cầu, ID nhiệm vụ: TO36
  - Tên nhiệm vụ: Phân tích sơ đồ dữ liệu (ERD), ID nhiệm vụ: TO37
  - Tên nhiệm vụ: Thu thập dữ liệu mẫu, ID nhiệm vụ: TO38
  - Tên nhiệm vụ: Cài đặt sơ đồ dữ liệu vào hệ thống, ID nhiệm vụ: TO40
  - Tên nhiệm vụ: Nhập dữ liệu mẫu, ID nhiệm vụ: TO41
  - Tên nhiệm vụ: Kiểm tra độ chính xác của dữ liệu (Data Test), ID nhiệm vụ: TO42
Thời gian thực thi truy vấn: 0.7065 giây


In [ ]:
get_full_member_details("AO13")


--- Tổng hợp thông tin chi tiết cho thành viên: AO13 ---

--- Thông tin thành viên --- 
ID: AO13
Họ Tên: Lâm Minh Thiện
Email: minhthien@gmail.com
Vai trò: member

--- Các nhóm thuộc về --- 
  - Tên nhóm: Nhóm 3, ID nhóm: BO3

--- Các dự án liên quan --- 
  - Tên dự án: Hệ thống quản lý nhà xe sinh viên, ID dự án: CO3

--- Các nhiệm vụ được giao --- 
  - Tên nhiệm vụ: Khảo sát dự án & Phân tích yêu cầu, ID nhiệm vụ: TO33
  - Tên nhiệm vụ: Phân tích sơ đồ dữ liệu (ERD), ID nhiệm vụ: TO34
  - Tên nhiệm vụ: Thu thập dữ liệu mẫu, ID nhiệm vụ: TO35
  - Tên nhiệm vụ: Cài đặt sơ đồ dữ liệu vào hệ thống, ID nhiệm vụ: TO37
  - Tên nhiệm vụ: Nhập dữ liệu mẫu, ID nhiệm vụ: TO38
  - Tên nhiệm vụ: Kiểm tra độ chính xác của dữ liệu (Data Test), ID nhiệm vụ: TO39
  - Tên nhiệm vụ: Triển khai dữ liệu & Tài liệu cho Nhóm Lập trình, ID nhiệm vụ: TO40


#truy vấn: tên dự án , công việc của dự án, người tham gia dự
#xem những người trong nhóm , công việc trên nhóm, nhưng trao đổi trên
#dự án có nhiều công việc nhất ,ít nhất
#nhóm nào có nhiều cv nhất, ít. Người đc giao cv nhiều nhất, ít nhất
#.
#Thống kê cv nào có nhiều trao đổi nhất, ít nhất


#Truy vấn: tên dự án , công việc của dự án, người tham gia dự

In [ ]:
def truyvan_namePr(project_name):
    print(f"\n--- Tổng hợp thông tin chi tiết cho dự án: {project_name} ---")
    projects_collection = db['projects']
    groups_collection = db['groups']
    tasks_collection = db['Tasks']
    project = projects_collection.find_one({"Name_pr": project_name})
    if project:
        print(f"\n--- Thông tin dự án ---")
        print(f"ID Dự án: {project.get('_id')}")
        print(f"Tên Dự án: {project.get('Name_pr')}")
        print(f"Mô tả: {project.get('Mo_ta')}")
        print(f"Ngày Bắt Đầu: {project.get('NgayBD').strftime('%Y-%m-%d') if project.get('NgayBD') else 'N/A'}")
        print(f"Ngày Kết Thúc: {project.get('NgayKT').strftime('%Y-%m-%d') if project.get('NgayKT') else 'N/A'}")
        group_id = project.get('group_id')
        if group_id:
            group = groups_collection.find_one({"_id": group_id})
            if group and 'Thanh_vien' in group:
                print(f"\n--- Thành viên tham gia dự án (nhóm {group.get('TenGr')} - ID: {group_id}) ---")
                for member in group['Thanh_vien']:
                    print(f"  - ID: {member.get('id')}, Họ Tên: {member.get('HoTen')}, Vai trò: {member.get('role_id')}")
        project_tasks = list(tasks_collection.find({"ID_DuAn": project.get('_id')}))
        if project_tasks:
            print(f"\n--- Các nhiệm vụ của dự án ---")
            for task in project_tasks:
                print(f"  - ID: {task.get('_id')}, Tên nhiệm vụ: {task.get('TenTask')}, Trạng thái: {task.get('Trangthai')}")
truyvan_namePr("Hệ thống quản lý ký túc xá")
#truyvan_namePr("Hệ thống quản lý nhà trọ")
#truyvan_namePr("Hệ thống quản lý nhà xe sinh viên")



--- Tổng hợp thông tin chi tiết cho dự án: Hệ thống quản lý ký túc xá ---

--- Thông tin dự án ---
ID Dự án: CO1
Tên Dự án: Hệ thống quản lý ký túc xá
Mô tả: Xây dựng phần mềm nhằm tự động hóa và tối ưu hóa các nghiệp vụ quản lý sinh viên, phòng ốc, thu chi và an ninh trong môi trường ký túc xá.
Ngày Bắt Đầu: 2025-12-06
Ngày Kết Thúc: 2026-01-10

--- Thành viên tham gia dự án (nhóm Nhóm 1 - ID: BO1) ---
  - ID: AO1, Họ Tên: Nguyễn An Ninh, Vai trò: member
  - ID: AO2, Họ Tên: Huỳnh Tân khoa, Vai trò: role_leader
  - ID: AO3, Họ Tên: Từ Gia Đạt, Vai trò: member
  - ID: AO4, Họ Tên: Thạch Minh Nhật, Vai trò: member
  - ID: AO5, Họ Tên: Đinh Hữu Nhân, Vai trò: member

--- Các nhiệm vụ của dự án ---
  - ID: TO1, Tên nhiệm vụ: Khảo sát dự án, Trạng thái: done
  - ID: TO2, Tên nhiệm vụ: Phân tích sơ đồ dữ liệu, Trạng thái: done
  - ID: TO3, Tên nhiệm vụ: Xác định các Ràng buộc & Quy tắc, Trạng thái: done
  - ID: TO4, Tên nhiệm vụ: Thu thập dữ liệu mẫu, Trạng thái: done
  - ID: TO5, Tên nhiệ

#Xem những người trong nhóm , công việc trên nhóm, những trao đổi
#Thống kê cv nào có nhiều trao đổi nhất, ít nhất


In [ ]:
def get_full_group_details(group_name):
    print(f"\n--- Tổng hợp thông tin chi tiết cho nhóm: {group_name} ---")
    groups_collection = db['groups']
    projects_collection = db['projects']
    tasks_collection = db['Tasks']
    comments_collection = db['Comments']
    group = groups_collection.find_one({"TenGr": group_name})
    if group:
        print("\n--- Thông tin nhóm ---")
        print(f"ID Nhóm: {group.get('_id')}")
        print(f"Tên Nhóm: {group.get('TenGr')}")
        print(f"Mô tả: {group.get('Mota_Gr')}")
        if 'Thanh_vien' in group and group['Thanh_vien']:
            print("\n--- Thành viên của nhóm ---")
            for member in group['Thanh_vien']:
                print(f"  - ID: {member.get('id')}, Họ Tên: {member.get('HoTen')}, Vai trò: {member.get('role_id')}")
        group_id = group.get('_id')
        if group_id:
            related_projects = list(projects_collection.find({"group_id": group_id}))
            if related_projects:
                print("\n--- Các dự án của nhóm ---")
                for project in related_projects:
                    print(f"  - Tên dự án: {project.get('Name_pr')}, ID dự án: {project.get('_id')}")
                    project_tasks = list(tasks_collection.find({"ID_DuAn": project.get('_id')}))
                    if project_tasks:
                        print(f"    --- Nhiệm vụ của dự án '{project.get('Name_pr')}' ---")
                        task_with_most_comments = None
                        max_comments = -1
                        task_with_least_comments = None
                        min_comments = float('inf')
                        for task in project_tasks:
                            comment_count = comments_collection.count_documents({"ID_Taks": task.get('_id')})
                            print(f"      - Tên nhiệm vụ: {task.get('TenTask')}, ID nhiệm vụ: {task.get('_id')}, Số bình luận: {comment_count}")
                            if comment_count > max_comments:
                                max_comments = comment_count
                                task_with_most_comments = task

                            if comment_count < min_comments:
                                min_comments = comment_count
                                task_with_least_comments = task

                        if task_with_most_comments:
                            print(f"    Nhiệm vụ có nhiều bình luận nhất: '{task_with_most_comments.get('TenTask')}' (ID: {task_with_most_comments.get('_id')}) với {max_comments} bình luận.")
                        if task_with_least_comments:
                            print(f"    Nhiệm vụ có ít bình luận nhất: '{task_with_least_comments.get('TenTask')}' (ID: {task_with_least_comments.get('_id')}) với {min_comments} bình luận.")

get_full_group_details("Nhóm 1")
get_full_group_details("Nhóm 2")
get_full_group_details("Nhóm 3")


--- Tổng hợp thông tin chi tiết cho nhóm: Nhóm 1 ---

--- Thông tin nhóm ---
ID Nhóm: BO1
Tên Nhóm: Nhóm 1
Mô tả: Xây dựng hệ thống quản lý ký túc xá sinh viên

--- Thành viên của nhóm ---
  - ID: AO1, Họ Tên: Nguyễn An Ninh, Vai trò: member
  - ID: AO2, Họ Tên: Huỳnh Tân khoa, Vai trò: role_leader
  - ID: AO3, Họ Tên: Từ Gia Đạt, Vai trò: member
  - ID: AO4, Họ Tên: Thạch Minh Nhật, Vai trò: member
  - ID: AO5, Họ Tên: Đinh Hữu Nhân, Vai trò: member

--- Các dự án của nhóm ---
  - Tên dự án: Hệ thống quản lý ký túc xá, ID dự án: CO1
    --- Nhiệm vụ của dự án 'Hệ thống quản lý ký túc xá' ---
      - Tên nhiệm vụ: Khảo sát dự án, ID nhiệm vụ: TO1, Số bình luận: 0
      - Tên nhiệm vụ: Phân tích sơ đồ dữ liệu, ID nhiệm vụ: TO2, Số bình luận: 0
      - Tên nhiệm vụ: Xác định các Ràng buộc & Quy tắc, ID nhiệm vụ: TO3, Số bình luận: 0
      - Tên nhiệm vụ: Thu thập dữ liệu mẫu, ID nhiệm vụ: TO4, Số bình luận: 0
      - Tên nhiệm vụ: Làm sạch dữ liệu, ID nhiệm vụ: TO5, Số bình luận: 0
    

#Dự án có nhiều công việc nhất ,ít nhất.


In [ ]:
def phan_tich_nhiem_vu_du_an():
    # In tieu de phan tich ra man hinh
    print("\n--- Số lượng nhiệm vụ cho dự án  ---")

    # Ket noi cac bo suu tap (collections) tu co so du lieu
    du_an_collection = db['projects']
    nhiem_vu_collection = db['Tasks']

    # Dung Aggregation Pipeline de nhom cac nhiem vu theo ID du an va dem so luong
    pipeline = [
        {
            # Nhom theo truong ID_DuAn va tinh tong so luong nhiem vu
            "$group": {
                "_id": "$ID_DuAn",
                "so_luong_nhiem_vu": {"$sum": 1}
            }
        },
        {
            # Sap xep theo so luong nhiem vu giam dan (-1)
            "$sort": {"so_luong_nhiem_vu": -1}
        }
    ]

    # Thuc thi truy van thong ke
    thong_ke_nhiem_vu = list(nhiem_vu_collection.aggregate(pipeline))

    if not thong_ke_nhiem_vu:
        print("Khong co nhiem vu nao duoc tim thay trong co so du lieu.")
        return

    # Chuyen doi ket qua sang DataFrame (Pandas) de de dang thao tac
    df_thong_ke = pd.DataFrame(thong_ke_nhiem_vu)
    df_thong_ke.rename(columns={'_id': 'id_du_an'}, inplace=True)

    # Xac dinh du an co nhieu nhiem vu nhat
    du_an_nhieu_nhat = df_thong_ke.loc[df_thong_ke['so_luong_nhiem_vu'].idxmax()]
    id_du_an_nhieu = du_an_nhieu_nhat['id_du_an']
    dem_nhieu_nhat = du_an_nhieu_nhat['so_luong_nhiem_vu']

    # Lay thong tin chi tiet (ten du an) tu collection projects
    info_du_an_nhieu = du_an_collection.find_one({"_id": id_du_an_nhieu})
    ten_du_an_nhieu = info_du_an_nhieu.get('Name_pr', 'Ten du an khong xac dinh') if info_du_an_nhieu else 'Khong tim thay du an'

    # Xac dinh du an co it nhiem vu nhat
    du_an_it_nhat = df_thong_ke.loc[df_thong_ke['so_luong_nhiem_vu'].idxmin()]
    id_du_an_it = du_an_it_nhat['id_du_an']
    dem_it_nhat = du_an_it_nhat['so_luong_nhiem_vu']

    # Lay thong tin chi tiet tu collection projects
    info_du_an_it = du_an_collection.find_one({"_id": id_du_an_it})
    ten_du_an_it = info_du_an_it.get('Name_pr', 'Ten du an khong xac dinh') if info_du_an_it else 'Khong tim thay du an'

    # In ket qua phan tich cuoi cung
    print(f"\n--- Kết quả --- ")
    print(f"Dự án có nhiều nhiệm vụ nhất: '{ten_du_an_nhieu}' (ID: {id_du_an_nhieu}) voi {dem_nhieu_nhat} nhiệm vụ.")
    print(f"Dự án có ít nhiệm vụ nhất: '{ten_du_an_it}' (ID: {id_du_an_it}) voi {dem_it_nhat} nhiệm vụ.")

phan_tich_nhiem_vu_du_an()


--- Số lượng nhiệm vụ cho dự án  ---

--- Ket qua phan tich --- 
Dự án có nhiều nhiệm vụ nhất: 'Hệ thống quản lý ký túc xá' (ID: CO1) voi 19 nhiệm vụ.
Dự án có ít nhiệm vụ nhất: 'Hệ thống quản lý nhà xe sinh viên' (ID: CO3) voi 14 nhiệm vụ.


#Nhóm nào có nhiều cv nhất, ít nhất.


In [ ]:
def phan_tich_nhiem_vu_nhom():

    print("\n--- Số lượng nhiệm vụ cho mỗi nhóm ---")

    # Ket noi cac bo suu tap (collections) tu co so du lieu
    cong_viec_collection = db['Tasks']
    du_an_collection = db['projects']
    nhom_collection = db['groups']

    # Thiet lap pipeline de truy van va thong ke du lieu tu nhieu bang
    pipeline = [
        {
            # Ket hop du lieu voi bang 'projects' dua tren ID du an
            "$lookup": {
                "from": "projects",
                "localField": "ID_DuAn",
                "foreignField": "_id",
                "as": "thong_tin_du_an"
            }
        },
        {
            # Trai phang mang thong tin du an de de dang truy cap thuoc tinh
            "$unwind": "$thong_tin_du_an"
        },
        {
            # Nhom du lieu theo ID nhom va dem tong so cong viec trong moi nhom
            "$group": {
                "_id": "$thong_tin_du_an.group_id",
                "so_luong_nhiem_vu": {"$sum": 1}
            }
        },
        {
            # Sap xep danh sach theo so luong cong viec giam dan
            "$sort": {"so_luong_nhiem_vu": -1}
        }
    ]

    # Thuc thi truy van thong ke va chuyen ket qua thanh danh sach
    thong_ke_nhom = list(cong_viec_collection.aggregate(pipeline))

    if not thong_ke_nhom:
        print("Khong co nhiem vu nao duoc tim thay hoac khong lien ket voi nhom.")
        return

    # Chuyen doi sang DataFrame de thao tac du lieu de dang hon
    df_thong_ke_nhom = pd.DataFrame(thong_ke_nhom)
    df_thong_ke_nhom.rename(columns={'_id': 'id_nhom'}, inplace=True)

    # Xac dinh nhom co nhieu nhiem vu nhat
    nhom_nhieu_nhat = df_thong_ke_nhom.loc[df_thong_ke_nhom['so_luong_nhiem_vu'].idxmax()]
    id_nhom_nhieu = nhom_nhieu_nhat['id_nhom']
    dem_nhieu = nhom_nhieu_nhat['so_luong_nhiem_vu']
    # Lay ten nhom tu bang 'groups'
    info_nhom_nhieu = nhom_collection.find_one({"_id": id_nhom_nhieu})
    ten_nhom_nhieu = info_nhom_nhieu.get('TenGr', 'Ten nhom khong xac dinh') if info_nhom_nhieu else 'Khong tim thay nhom'

    # Xac dinh nhom co it nhiem vu nhat
    nhom_it_nhat = df_thong_ke_nhom.loc[df_thong_ke_nhom['so_luong_nhiem_vu'].idxmin()]
    id_nhom_it = nhom_it_nhat['id_nhom']
    dem_it = nhom_it_nhat['so_luong_nhiem_vu']
    # Lay ten nhom tu bang 'groups'
    info_nhom_it = nhom_collection.find_one({"_id": id_nhom_it})
    ten_nhom_it = info_nhom_it.get('TenGr', 'Ten nhom khong xac dinh') if info_nhom_it else 'Khong tim thay nhom'

    # In ket qua phan tich cuoi cung
    print("\n--- Ket qua phan tich --- ")
    print(f"Nhóm có nhiều nhiệm vụ nhất: '{ten_nhom_nhieu}' (ID: {id_nhom_nhieu}) voi {dem_nhieu} nhiem vu.")
    print(f"Nhóm có ít nhiệm vụ nhất: '{ten_nhom_it}' (ID: {id_nhom_it}) voi {dem_it} nhiem vu.")

phan_tich_nhiem_vu_nhom()


--- So luong nhiem vu cho moi nhom ---

--- Ket qua phan tich --- 
Nhom co nhieu nhiem vu nhat: 'Nhóm 1' (ID: BO1) voi 19 nhiem vu.
Nhom co it nhiem vu nhat: 'Nhóm 3' (ID: BO3) voi 14 nhiem vu.


In [ ]:
def phan_tich_nhiem_vu_nhom():

    print("\n--- Số lượng nhiệm vụ cho mỗi nhóm ---")

    # Ket noi cac bo suu tap (collections) tu co so du lieu
    cong_viec_collection = db['Tasks']
    du_an_collection = db['projects']
    nhom_collection = db['groups']

    # Thiet lap pipeline de truy van va thong ke du lieu tu nhieu bang
    pipeline = [
        {
            # Ket hop du lieu voi bang 'projects' dua tren ID du an
            "$lookup": {
                "from": "projects",
                "localField": "ID_DuAn",
                "foreignField": "_id",
                "as": "thong_tin_du_an"
            }
        },
        {
            # Trai phang mang thong tin du an de de dang truy cap thuoc tinh
            "$unwind": "$thong_tin_du_an"
        },
        {
            # Nhom du lieu theo ID nhom va dem tong so cong viec trong moi nhom
            "$group": {
                "_id": "$thong_tin_du_an.group_id",
                "so_luong_nhiem_vu": {"$sum": 1}
            }
        },
        {
            # Sap xep danh sach theo so luong cong viec giam dan
            "$sort": {"so_luong_nhiem_vu": -1}
        }
    ]

    # Thuc thi truy van thong ke va chuyen ket qua thanh danh sach
    thong_ke_nhom = list(cong_viec_collection.aggregate(pipeline))

    if not thong_ke_nhom:
        print("Khong co nhiem vu nao duoc tim thay hoac khong lien ket voi nhom.")
        return

    # Chuyen doi sang DataFrame de thao tac du lieu de dang hon
    df_thong_ke_nhom = pd.DataFrame(thong_ke_nhom)
    df_thong_ke_nhom.rename(columns={'_id': 'id_nhom'}, inplace=True)

    # Xac dinh nhom co nhieu nhiem vu nhat
    nhom_nhieu_nhat = df_thong_ke_nhom.loc[df_thong_ke_nhom['so_luong_nhiem_vu'].idxmax()]
    id_nhom_nhieu = nhom_nhieu_nhat['id_nhom']
    dem_nhieu = nhom_nhieu_nhat['so_luong_nhiem_vu']
    # Lay ten nhom tu bang 'groups'
    info_nhom_nhieu = nhom_collection.find_one({"_id": id_nhom_nhieu})
    ten_nhom_nhieu = info_nhom_nhieu.get('TenGr', 'Ten nhom khong xac dinh') if info_nhom_nhieu else 'Khong tim thay nhom'

    # Xac dinh nhom co it nhiem vu nhat
    nhom_it_nhat = df_thong_ke_nhom.loc[df_thong_ke_nhom['so_luong_nhiem_vu'].idxmin()]
    id_nhom_it = nhom_it_nhat['id_nhom']
    dem_it = nhom_it_nhat['so_luong_nhiem_vu']
    # Lay ten nhom tu bang 'groups'
    info_nhom_it = nhom_collection.find_one({"_id": id_nhom_it})
    ten_nhom_it = info_nhom_it.get('TenGr', 'Ten nhom khong xac dinh') if info_nhom_it else 'Khong tim thay nhom'

    # In ket qua phan tich cuoi cung
    print("\n--- Ket qua phan tich --- ")
    print(f"Nhóm có nhiều nhiệm vụ nhất: '{ten_nhom_nhieu}' (ID: {id_nhom_nhieu}) voi {dem_nhieu} nhiem vu.")
    print(f"Nhóm có ít nhiệm vụ nhất: '{ten_nhom_it}' (ID: {id_nhom_it}) voi {dem_it} nhiem vu.")

phan_tich_nhiem_vu_nhom()


--- So luong nhiem vu cho moi nhom ---

--- Ket qua phan tich --- 
Nhom co nhieu nhiem vu nhat: 'Nhóm 1' (ID: BO1) voi 19 nhiem vu.
Nhom co it nhiem vu nhat: 'Nhóm 3' (ID: BO3) voi 14 nhiem vu.


#Người đc giao công việc nhiều nhất, ít nhất.

In [ ]:
def phan_tich_nhiem_vu_thanh_vien_theo_nhom():
    print("\n--- Phan tich so luong nhiem vu duoc giao cho moi thanh vien trong tung nhom ---")

    # Ket noi cac collections tu co so du lieu
    nhom_collection = db['groups']
    du_an_collection = db['projects']
    cong_viec_collection = db['Tasks']
    thanh_vien_collection = db['Thanh_vien']

    # Lay danh sach tat ca cac nhom
    tat_ca_nhom = list(nhom_collection.find({}))

    if not tat_ca_nhom:
        print("Khong tim thay nhom nao.")
        return

    for nhom in tat_ca_nhom:
        id_nhom = nhom.get('_id')
        ten_nhom = nhom.get('TenGr', 'Khong xac dinh')
        # Lay tap hop ID cua cac thanh vien thuoc nhom hien tai
        ds_thanh_vien_nhom = {tv.get('id') for tv in nhom.get('Thanh_vien', [])}

        print(f"\n--- {ten_nhom} (ID: {id_nhom}) ---")

        # 1. Tim cac du an thuoc ve nhom hien tai
        du_an_cua_nhom = list(du_an_collection.find({"group_id": id_nhom}))
        if not du_an_cua_nhom:
            print("  Khong tim thay du an nao cho nhom nay.")
            continue

        id_cac_du_an = [da.get('_id') for da in du_an_cua_nhom]

        # 2. Tim cac nhiem vu thuoc cac du an nay
        ds_cong_viec_nhom = list(cong_viec_collection.find({"ID_DuAn": {"$in": id_cac_du_an}}))

        if not ds_cong_viec_nhom:
            print("  Khong tim thay nhiem vu nao cho cac du an cua nhom nay.")
            continue

        # Khoi tao bo dem nhiem vu cho moi thanh vien trong nhom bang 0
        dem_nhiem_vu_thanh_vien = {id_tv: 0 for id_tv in ds_thanh_vien_nhom}

        # 3. Dem so nhiem vu cho tung thanh vien trong nhom
        for cong_viec in ds_cong_viec_nhom:
            chuoi_id_thanh_vien = cong_viec.get('ID_ThanhVien')
            if chuoi_id_thanh_vien:
                # Tach chuoi ID bang dau phay, loai bo khoang trang va chi giu lai thanh vien thuoc nhom nay
                thanh_vien_duoc_giao = [m.strip() for m in chuoi_id_thanh_vien.split(',') if m.strip() in ds_thanh_vien_nhom]
                for id_tv in thanh_vien_duoc_giao:
                    dem_nhiem_vu_thanh_vien[id_tv] += 1

        # Loc ra nhung thanh vien co it nhat 1 nhiem vu
        thanh_vien_co_viec = {id_tv: count for id_tv, count in dem_nhiem_vu_thanh_vien.items() if count > 0}

        if not thanh_vien_co_viec:
            print("  Khong co thanh vien nao trong nhom nay duoc giao nhiem vu cu the.")
            continue

        # 4. Xac dinh thanh vien co nhieu nhiem vu nhat va it nhiem vu nhat
        if thanh_vien_co_viec:
            # Tim nguoi nhieu viec nhat
            id_tv_nhieu_nhat = max(thanh_vien_co_viec, key=thanh_vien_co_viec.get)
            so_luong_nhieu_nhat = thanh_vien_co_viec[id_tv_nhieu_nhat]

            # Bao gom ca nhung thanh vien co 0 nhiem vu de tim nguoi it viec nhat
            tat_ca_thong_ke_nhom = {id_tv: dem_nhiem_vu_thanh_vien.get(id_tv, 0) for id_tv in ds_thanh_vien_nhom}

            id_tv_it_nhat = min(tat_ca_thong_ke_nhom, key=tat_ca_thong_ke_nhom.get)
            so_luong_it_nhat = tat_ca_thong_ke_nhom[id_tv_it_nhat]

            # Lay thong tin ten thanh vien tu collection Thanh_vien
            info_tv_nhieu = thanh_vien_collection.find_one({"_id": id_tv_nhieu_nhat})
            ten_tv_nhieu = info_tv_nhieu.get('HoTen', 'Khong xac dinh') if info_tv_nhieu else 'Khong xac dinh'

            info_tv_it = thanh_vien_collection.find_one({"_id": id_tv_it_nhat})
            ten_tv_it = info_tv_it.get('HoTen', 'Khong xac dinh') if info_tv_it else 'Khong xac dinh'

            print(f"  Thanh vien duoc giao nhieu nhiem vu nhat: '{ten_tv_nhieu}' (ID: {id_tv_nhieu_nhat}) voi {so_luong_nhieu_nhat} nhiem vu.")
            print(f"  Thanh vien duoc giao it nhiem vu nhat: '{ten_tv_it}' (ID: {id_tv_it_nhat}) voi {so_luong_it_nhat} nhiem vu.")
        else:
            print("  Khong co thanh vien nao trong nhom nay duoc giao nhiem vu.")

phan_tich_nhiem_vu_thanh_vien_theo_nhom()


--- Phan tich so luong nhiem vu duoc giao cho moi thanh vien trong tung nhom ---

--- Nhóm 1 (ID: BO1) ---
  Thanh vien duoc giao nhieu nhiem vu nhat: 'Huỳnh Tân khoa' (ID: AO2) voi 9 nhiem vu.
  Thanh vien duoc giao it nhiem vu nhat: 'Thạch Minh Nhật' (ID: AO4) voi 4 nhiem vu.

--- Nhóm 2 (ID: BO2) ---
  Thanh vien duoc giao nhieu nhiem vu nhat: 'Trần Hoàng Long' (ID: AO6) voi 6 nhiem vu.
  Thanh vien duoc giao it nhiem vu nhat: 'Trần Phú Dinh' (ID: AO8) voi 4 nhiem vu.

--- Nhóm 3 (ID: BO3) ---
  Thanh vien duoc giao nhieu nhiem vu nhat: 'Đinh Minh Vẹn' (ID: AO14) voi 6 nhiem vu.
  Thanh vien duoc giao it nhiem vu nhat: 'Huỳnh Thiên Sơn' (ID: AO11) voi 4 nhiem vu.
